In [1]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv(r"E:\NULLCLASS\datasets\cleaned_data.csv")

df["Revenue"] = df["Price"] * df["Installs"]

# Group by Category and Type, compute average installs & revenue
category_summary = df.groupby(["Category", "Type"]).agg(
    Avg_Installs=("Installs", "mean"),
    Avg_Revenue=("Revenue", "mean")
).reset_index()

# Create dual-axis chart
fig = go.Figure()

# Bar chart for average installs
fig.add_trace(go.Bar(x=category_summary["Category"], y=category_summary["Avg_Installs"], 
                     name="Average Installs", marker_color="blue"))

# Line chart for average revenue
fig.add_trace(go.Scatter(x=category_summary["Category"], y=category_summary["Avg_Revenue"], 
                         name="Average Revenue", marker_color="red", mode="lines+markers"))

fig.update_layout(title="Comparison of Average Installs and Revenue (Free vs Paid)",
                  xaxis_title="Category", yaxis_title="Count",
                  legend_title="Metric")

fig.write_html("Revenue_Graph_7.html")
print("Graph saved as 'Revenue_Graph_7.html'")


Graph saved as 'Revenue_Graph_7.html'
